# 신용카드 거래 사기 여부 예측하기

신용카드 거래 관련 정보(거래의 사기 여부 정보 포함)를 바탕으로 사기여부를 예측하는 머신러닝 모델을 만들어보자. 훈련용 데이터와 테스트 데이터가 별도로 제공되며 훈련용 데이터로 모델을 만들고 테스트용 데이터를 만든 모델의 입력으로 지정하여 나온 결과를 최종 제출합니다.

최종 제출물의 평가는 F1 점수로 이뤄지는데 그 이유는 이 데이터셋의 경우 정상 데이터와 사기 데이터의 비율이 굉장히 왜곡되어 있기 때문입니다. 정상 데이터가 훨씬 더 크며 이런 경우 정확도는 좋은 지표가 되지 않고 F1 점수가 더 좋은 지표가 됩니다. 실제 현업에서는 이렇게 비율이 왜곡된 데이터셋이 많습니다. 

## 훈련/테스트 데이터 셋 설명

이 데이터셋은 2013년 유럽연합 신용카드 사용자들의 실제 거래로부터 만들어졌고 거래의 대부분이 정상 거래입니다. 개인정보 보호 이슈 때문에 거래 시간과 금액를 제외한 나머지 피쳐들은 PCA 테크닉을 통해 이미 스케일된 형태로 제공되며 NULL인 필드들도 존재하지 않습니다. 이미 PCA를 거쳐서 변환된 피쳐들의 경우 그 의미를 설명할 수 없다는 점 미리 밝힙니다. 다음과 같은 2개의 파일들이 제공됩니다.

* train.csv
* test.csv

앞서 섹션에서 언급한 모델을 만들기 위한 훈련용 데이터(train.csv)와 나중에 만들어진 모델의 성능을 측정하기 위한 테스트 데이터(test.csv)가 제공이 됩니다. 두 개의 포맷은 전자에는 레이블 필드가 있고 후자에는 없고 대신 일련번호 필드가 존재한다는 점을 제외하면 동일합니다.

### Feature 설명

먼저 훈련용 데이터와 테스트용 데이터에 들어오는 Feature들을 정리해보면 다음과 같습니다.

|필드이름|	타입|	설명|
|:-|:-|:-|
|Time|	정수|	이 데이터셋에서 처음 거래와 이 거래 간의 시간 차이를 초단위로 표현한 정수로 가장 처음 거래는 이 필드의 값이 0이 됩니다.|
|V1부터 V28|	실수|	PCA를 통해 이미 1과 -1사이로 스케일된 28개의 피쳐가 제공됩니다.|
|Amount|실수|	신용카드 거래 금액을 나타냅니다.|
|Class|	정수|	거래의 사기 여부를 나타내는 값이 되며 0이면 정상 거래이고 1이면 사기 거래임을 나타냅니다. 이 필드는 test.csv에는 들어오지 않습니다.|

### 훈련용 데이터 (train.csv) 예제

신용카드 거래의 사기 여부를 훈련하기 위한 데이터는 train.csv 파일에 있고 총 31개의 필드로 구성되어 있고 그 중의 마지막 하는 사기여부를 나타내는 레이블 필드입니다. 훈련용 데이터 파일에는 총 250K개의 레코드(와 하나의 헤더)가 제공되며 참고로 처음 2줄(첫 줄은 헤더)은 다음과 같습니다.

|Time|V1|V2|V3|V4|V5|V6|V7|V8|V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|0|-1.359807134|-0.072781173|2.536346738|1.378155224|-0.33832077|0.462387778|0.239598554|0.098697901|0.36378697|0.090794172|-0.551599533|-0.617800856|-0.991389847|-0.311169354|1.468176972|-0.470400525|0.207971242|0.02579058|0.40399296|0.251412098|-0.018306778|0.277837576|-0.11047391|0.066928075|0.128539358|-0.189114844|0.133558377|-0.021053053|149.62|0|

### 테스트용 데이터 (test.csv) 예제

앞서 만든 모델로 풀어야 하는 문제들이 들어있는 파일들이 바로 test.csv입니다. 이 파일의 구성은 앞서 train.csv와 비슷하게 한 줄의 헤더 라인과 30K개의 feature 데이터 라인들로 구성되어 있습니다. test.csv에는 1) train.csv와는 달리 사기 여부를 나타내는 class 필드가 없고 2) 각 레코드를 식별해주는 ID필드가 존재한다는 점을 제외하고는 train.csv와 동일합니다. 예를 들어 처음 2줄(첫 줄은 헤더)은 다음과 같습니다.

|ID|Time|V1|V2|V3|V4|V5|V6|V7|V8|V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|1|0|-1.359807134|-0.072781173|2.536346738|1.378155224|-0.33832077|0.462387778|0.239598554|0.098697901|0.36378697|0.090794172|-0.551599533|-0.617800856|-0.991389847|-0.311169354|1.468176972|-0.470400525|0.207971242|0.02579058|0.40399296|0.251412098|-0.018306778|0.277837576|-0.11047391|0.066928075|0.128539358|-0.189114844|0.133558377|-0.021053053|149.62|

## 초기코드 선택

Python, R 중 본인의 선호 언어에 따라 초기 코드를 선택하세요.
Python을 선호한다면 Python 초기 코드를, R을 선호한다면 R 초기 코드만 남기면 됩니다.

In [1]:
# 초기코드 - python
import pandas as pd

# 데이터 로드
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# 초기코드 - R

# 데이터 로드
train <- read.csv('./data/train.csv')
test <- read.csv('./data/test.csv')

head(train)

In [2]:
x_features_list = train.columns[0:-1]
x_features = train[x_features_list]

In [3]:
x_features.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [4]:
y_labels = train.Class

In [5]:
test.head()

,ID,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,1,154675,-3.484268,-3.478996,-1.997164,5.583060,1.277970,-0.257365,1.571584,0.371628,...,2.627661,1.040880,0.547391,1.763899,0.698150,-0.943907,0.204003,0.203824,-0.181849,722.58
1,2,154676,-0.162076,0.275857,1.609756,0.238198,-0.506164,0.332477,-0.232509,0.268916,...,0.022396,-0.031401,0.453633,0.056230,-0.019463,-1.131644,0.512097,0.276639,0.264650,26.50
2,3,154677,2.046965,-0.144759,-1.676612,0.005863,0.539168,-0.215378,0.065013,-0.116352,...,-0.137020,0.275938,0.857349,-0.073066,0.132017,0.324705,-0.118361,-0.028546,-0.063637,19.95
3,4,154677,2.095883,-1.043785,-1.134008,-1.139906,-0.552036,-0.216809,-0.766197,0.071149,...,0.015997,-0.229421,-0.976831,0.433483,0.178759,-0.573893,-0.714417,-0.040165,-0.046876,56.90
4,5,154678,2.277383,-0.616234,-1.603675,-1.053846,-0.158831,-0.984861,-0.238083,-0.453934,...,0.066643,0.397443,1.230527,-0.121151,-0.622204,0.343449,0.109208,-0.032566,-0.070512,15.00


In [6]:
X_test = test[test.columns[1:]]
X_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,154675,-3.484268,-3.478996,-1.997164,5.583060,1.277970,-0.257365,1.571584,0.371628,-2.794352,...,2.627661,1.040880,0.547391,1.763899,0.698150,-0.943907,0.204003,0.203824,-0.181849,722.58
1,154676,-0.162076,0.275857,1.609756,0.238198,-0.506164,0.332477,-0.232509,0.268916,1.316526,...,0.022396,-0.031401,0.453633,0.056230,-0.019463,-1.131644,0.512097,0.276639,0.264650,26.50
2,154677,2.046965,-0.144759,-1.676612,0.005863,0.539168,-0.215378,0.065013,-0.116352,0.387979,...,-0.137020,0.275938,0.857349,-0.073066,0.132017,0.324705,-0.118361,-0.028546,-0.063637,19.95
3,154677,2.095883,-1.043785,-1.134008,-1.139906,-0.552036,-0.216809,-0.766197,0.071149,-0.321957,...,0.015997,-0.229421,-0.976831,0.433483,0.178759,-0.573893,-0.714417,-0.040165,-0.046876,56.90
4,154678,2.277383,-0.616234,-1.603675,-1.053846,-0.158831,-0.984861,-0.238083,-0.453934,-0.657152,...,0.066643,0.397443,1.230527,-0.121151,-0.622204,0.343449,0.109208,-0.032566,-0.070512,15.00


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf.fit(x_features, y_labels)
preds = rf.predict(X_test)

In [ ]:
preds

In [ ]:
test['Class'] = preds
test.head()

In [ ]:
drop_list = test.columns[1:-1]
drop_list

In [ ]:
dt = test.drop(drop_list, axis=1)
dt.head()


### 최종 제출 파일

여러분이 제출해야하는 파일은 다음과 같은 두 개의 필드는 갖는 CSV 파일입니다.
 
1. ID
2. Class


ID는 test.csv에서 받은 ID 필드의 값을 사용합니다. Class는 해당 거래의 사기 여부 예측 값에 해당합니다. 예를 들면 아래와 같습니다.

|ID|Class|
|-|-|
|1|0|
|2|0|
|3|0|
|4|1|
|5|1|
|...|...|

앞서 언급했던 것처럼 보통 분류 지도학습의 경우 훈련용 데이터가 제공되고 예측해야하는 레이블 정보가 어느 정도 밸런스를 유지해야 합니다.  
이 문제처럼 사기여부를 예측하는 이진 분류기라면 보통 정상 데이터의 비율이 훨씬 더 높고 사기 데이터의 비율이 상대적으로 낮게 나오게 됩니다.  
그래서 최종 성능은 정확도 (accuracy)로 측정하지 않고 F1 점수로 측정합니다. 


##### 3. 결과 저장

채점을 위해, 위에서 구한 데이터를 현재 파일과 같은 디렉토리(.ipynb 파일이 있는 디렉토리)에 `submission.csv`이라는 이름으로 저장해주세요.

In [ ]:
# 여기에 코드를 작성하세요.

In [ ]:
# csv 파일 저장 예시 - python
dt.to_csv('submission.csv')

In [ ]:
# csv 파일 저장 예시 - R
write.csv(df, file='submission.csv')